In [2]:
# modelo baseado no modelo de OLACEFS_DAM
# https://github.com/edemir-matcomp/OLACEFS_DAM/blob/master/4_Change_Detection/models.py

from keras.models import Model
from keras.layers import (
    Input, 
    concatenate, 
    Conv2D, 
    MaxPooling2D, 
    Conv2DTranspose, 
    BatchNormalization, 
    Activation, 
    ConvLSTM2D, 
    UpSampling2D, 
    Reshape, 
    Dropout
)  
import numpy as np

def unet_lstm2(input_size = (256,256,1), n_filters_start=16, growth_factor=2, droprate=0.2):
    N = input_size[0]
    inputs = Input(input_size)

    kernel_size = 3
    activation = 'relu'
    
    #down sampling
    n_filters = n_filters_start
    conv1 = Conv2D(n_filters, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = BatchNormalization() (conv1)
    conv1 = Dropout(droprate/2) (conv1)
    conv1 = Conv2D(n_filters, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(conv1)
    conv1 = BatchNormalization() (conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    n_filters *= growth_factor
    conv2 = Conv2D(n_filters, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = BatchNormalization() (conv2)
    conv2 = Dropout(droprate/2) (conv2)
    conv2 = Conv2D(n_filters, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(conv2)
    conv2 = BatchNormalization() (conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    n_filters *= growth_factor
    conv3 = Conv2D(n_filters, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = BatchNormalization() (conv3)
    conv3 = Dropout(droprate) (conv3)
    conv3 = Conv2D(n_filters, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(conv3)
    conv3 = BatchNormalization() (conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    n_filters *= growth_factor
    conv4 = Conv2D(n_filters, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = BatchNormalization() (conv4)
    conv4 = Dropout(droprate) (conv4)
    conv4 = Conv2D(n_filters, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(conv4)
    conv4 = BatchNormalization() (conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2)) (conv4)
    
    # up sampling
    # up5
    n_filters //= growth_factor
    up5 = Conv2DTranspose(n_filters, kernel_size=2, strides=2, padding='same',kernel_initializer = 'he_normal')(conv4)
    up5 = BatchNormalization(axis=3)(up5)
    up5 = Activation('relu')(up5)

    x1 = Reshape(target_shape=(1, np.int32(N/4), np.int32(N/4), n_filters))(conv3)
    x2 = Reshape(target_shape=(1, np.int32(N/4), np.int32(N/4), n_filters))(up5)

    merge5  = concatenate([x1,x2], axis = 1) 
    merge5 = ConvLSTM2D(filters = n_filters//growth_factor, kernel_size=(3, 3), padding='same', return_sequences = False, go_backwards = True,kernel_initializer = 'he_normal' )(merge5)
            
    conv5 = Conv2D(n_filters, kernel_size, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(merge5)
    conv5 = BatchNormalization() (conv5)
    conv5 = Dropout(droprate) (conv5)
    conv5 = Conv2D(n_filters, kernel_size, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(conv5)
    conv5 = BatchNormalization() (conv5)
    
    # up 6
    n_filters //= growth_factor
    up6 = Conv2DTranspose(n_filters, kernel_size=2, strides=2, padding='same',kernel_initializer = 'he_normal')(conv5)
    up6 = BatchNormalization(axis=3)(up6)
    up6 = Activation(activation)(up6)

    x1 = Reshape(target_shape=(1, np.int32(N/2), np.int32(N/2), n_filters))(conv2)
    x2 = Reshape(target_shape=(1, np.int32(N/2), np.int32(N/2), n_filters))(up6)

    merge6  = concatenate([x1,x2], axis = 1) 
    merge6 = ConvLSTM2D(filters = n_filters//growth_factor, kernel_size=(3, 3), padding='same', return_sequences = False, go_backwards = True,kernel_initializer = 'he_normal' )(merge6)
            
    conv6 = Conv2D(n_filters, kernel_size, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = BatchNormalization() (conv6)
    conv6 = Dropout(droprate) (conv6)
    conv6 = Conv2D(n_filters, kernel_size, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(conv6)
    conv6 = BatchNormalization() (conv6)
    
    # up 7
    n_filters //= growth_factor
    up7 = Conv2DTranspose(n_filters, kernel_size=2, strides=2, padding='same',kernel_initializer = 'he_normal')(conv6)
    up7 = BatchNormalization(axis=3)(up7)
    up7 = Activation(activation)(up7)
   
    x1 = Reshape(target_shape=(1, N, N, n_filters))(conv1)
    x2 = Reshape(target_shape=(1, N, N, n_filters))(up7)

    merge7  = concatenate([x1,x2], axis = 1) 
    merge7 = ConvLSTM2D(filters = n_filters//growth_factor, kernel_size=(3, 3), padding='same', return_sequences = False, go_backwards = True,kernel_initializer = 'he_normal' )(merge7)
            
    conv7 = Conv2D(n_filters, kernel_size, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = BatchNormalization() (conv7)
    conv7 = Dropout(droprate) (conv7)
    conv7 = Conv2D(n_filters, kernel_size, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(conv7)
    conv7 = BatchNormalization() (conv7)
    
    #conv8 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
    #conv9 = Conv2D(1, 1, activation = 'sigmoid')(conv8)

    outputs = Conv2D(1, 1, activation='sigmoid') (conv7)

    model = Model(inputs=[inputs], outputs=[outputs])
    
    return model

def unet_lstm(input_size = (256,256,1), n_filters_start=16, growth_factor=2, droprate=0.2):
    N = input_size[0]
    inputs = Input(input_size)
    
    n_filters = n_filters_start
    conv1 = Conv2D(n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    n_filters *= growth_factor
    conv2 = Conv2D(n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    n_filters *= growth_factor
    conv3 = Conv2D(n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    conv3 = Dropout(droprate)(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    n_filters *= growth_factor
    conv4 = Conv2D(n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)     
    conv4_1 = Conv2D(n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    conv4_1 = Dropout(droprate)(conv4_1)

    n_filters //= growth_factor
    up6 = Conv2DTranspose(n_filters, kernel_size=2, strides=2, padding='same',kernel_initializer = 'he_normal')(conv4_1)
    up6 = BatchNormalization(axis=3)(up6)
    up6 = Activation('relu')(up6)

    x1 = Reshape(target_shape=(1, np.int32(N/4), np.int32(N/4), n_filters))(conv3)
    x2 = Reshape(target_shape=(1, np.int32(N/4), np.int32(N/4), n_filters))(up6)

    merge6  = concatenate([x1,x2], axis = 1) 
    merge6 = ConvLSTM2D(filters = n_filters//growth_factor, kernel_size=(3, 3), padding='same', return_sequences = False, go_backwards = True,kernel_initializer = 'he_normal' )(merge6)
            
    conv6 = Conv2D(n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    n_filters //= growth_factor
    up7 = Conv2DTranspose(n_filters, kernel_size=2, strides=2, padding='same',kernel_initializer = 'he_normal')(conv6)
    up7 = BatchNormalization(axis=3)(up7)
    up7 = Activation('relu')(up7)

    x1 = Reshape(target_shape=(1, np.int32(N/2), np.int32(N/2), n_filters))(conv2)
    x2 = Reshape(target_shape=(1, np.int32(N/2), np.int32(N/2), n_filters))(up7)
    merge7  = concatenate([x1,x2], axis = 1) 
    merge7 = ConvLSTM2D(filters = n_filters//growth_factor, kernel_size=(3, 3), padding='same', return_sequences = False, go_backwards = True,kernel_initializer = 'he_normal' )(merge7)
        
    conv7 = Conv2D(n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    n_filters //= growth_factor
    up8 = Conv2DTranspose(n_filters, kernel_size=2, strides=2, padding='same',kernel_initializer = 'he_normal')(conv7)
    up8 = BatchNormalization(axis=3)(up8)
    up8 = Activation('relu')(up8)    

    x1 = Reshape(target_shape=(1, N, N, n_filters))(conv1)
    x2 = Reshape(target_shape=(1, N, N, n_filters))(up8)
    merge8  = concatenate([x1,x2], axis = 1) 
    merge8 = ConvLSTM2D(filters = n_filters//growth_factor, kernel_size=(3, 3), padding='same', return_sequences = False, go_backwards = True,kernel_initializer = 'he_normal' )(merge8)    
    
    conv8 = Conv2D(n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
    conv8 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
    conv9 = Conv2D(1, 1, activation = 'sigmoid')(conv8)

    model = Model(inputs, conv9)

    return model

def unet2(input_size = (256,256,1)):
    inputs = Input(input_size)

    kernel_size = (3, 3)
    activation = 'relu'

    conv1 = Conv2D(32, kernel_size, activation=activation, padding='same') (inputs)
    conv1 = BatchNormalization() (conv1)
    conv1 = Dropout(0.1) (conv1)
    conv1 = Conv2D(32, kernel_size, activation=activation, padding='same') (conv1)
    conv1 = BatchNormalization() (conv1)
    pooling1 = MaxPooling2D((2, 2)) (conv1)

    conv2 = Conv2D(64, kernel_size, activation=activation, padding='same') (pooling1)
    conv2 = BatchNormalization() (conv2)
    conv2 = Dropout(0.1) (conv2)
    conv2 = Conv2D(64, kernel_size, activation=activation, padding='same') (conv2)
    conv2 = BatchNormalization() (conv2)
    pooling2 = MaxPooling2D((2, 2)) (conv2)

    conv3 = Conv2D(128, kernel_size, activation=activation, padding='same') (pooling2)
    conv3 = BatchNormalization() (conv3)
    conv3 = Dropout(0.2) (conv3)
    conv3 = Conv2D(128, kernel_size, activation=activation, padding='same') (conv3)
    conv3 = BatchNormalization() (conv3)
    pooling3 = MaxPooling2D((2, 2)) (conv3)

    conv4 = Conv2D(256, kernel_size, activation=activation, padding='same') (pooling3)
    conv4 = BatchNormalization() (conv4)
    conv4 = Dropout(0.2) (conv4)
    conv4 = Conv2D(256, kernel_size, activation=activation, padding='same') (conv4)
    conv4 = BatchNormalization() (conv4)
    pooling4 = MaxPooling2D(pool_size=(2, 2)) (conv4)

    conv5 = Conv2D(512, kernel_size, activation=activation, padding='same') (pooling4)
    conv5 = BatchNormalization() (conv5)
    conv5 = Dropout(0.3) (conv5)
    conv5 = Conv2D(512, kernel_size, activation=activation, padding='same') (conv5)
    conv5 = BatchNormalization() (conv5)

    upsample6 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same') (conv5)
    upsample6 = concatenate([upsample6, conv4])
    conv6 = Conv2D(256, kernel_size, activation=activation, padding='same') (upsample6)
    conv6 = BatchNormalization() (conv6)
    conv6 = Dropout(0.2) (conv6)
    conv6 = Conv2D(256, kernel_size, activation=activation, padding='same') (conv6)
    conv6 = BatchNormalization() (conv6)

    upsample7 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (conv6)
    upsample7 = concatenate([upsample7, conv3])
    conv7 = Conv2D(128, kernel_size, activation=activation, padding='same') (upsample7)
    conv7 = BatchNormalization() (conv7)
    conv7 = Dropout(0.2) (conv7)
    conv7 = Conv2D(128, kernel_size, activation=activation, padding='same') (conv7)
    conv7 = BatchNormalization() (conv7)

    upsample8 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (conv7)
    upsample8 = concatenate([upsample8, conv2])
    conv8 = Conv2D(64, kernel_size, activation=activation, padding='same') (upsample8)
    conv8 = BatchNormalization() (conv8)
    conv8 = Dropout(0.1) (conv8)
    conv8 = Conv2D(64, kernel_size, activation=activation, padding='same') (conv8)
    conv8 = BatchNormalization() (conv8)

    upsample9 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (conv8)
    upsample9 = concatenate([upsample9, conv1], axis=3)
    conv9 = Conv2D(32, kernel_size, activation=activation, padding='same') (upsample9)
    conv9 = BatchNormalization() (conv9)
    conv9 = Dropout(0.1) (conv9)
    conv9 = Conv2D(32, kernel_size, activation=activation, padding='same') (conv9)
    conv9 = BatchNormalization() (conv9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid') (conv9)

    model = Model(inputs=[inputs], outputs=[outputs])
    
    return model
     
def unet(input_size = (256,256,1)):
    inputs = Input(input_size)
    print(inputs.shape)
    conv1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    conv3 = Dropout(0.5)(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)

    up6 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop4))
    merge6 = concatenate([conv3,up6], axis = 3)
    conv6 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv2,up7], axis = 3)
    conv7 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(32, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv1,up8], axis = 3)
    conv8 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
    conv8 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
    conv9 = Conv2D(1, 1, activation = 'sigmoid')(conv8)

    model = Model(inputs, conv9)

    return model
    